# IMPORT LIBRARIES

In [1]:
from urllib import request
import xml.etree.ElementTree as ET
import pandas as pd
import sqlite3
from datetime import datetime

# GET XML FROM URL

In [2]:
url = 'https://www.tcmb.gov.tr/kurlar/today.xml'
print ("URL:", url)

URL: https://www.tcmb.gov.tr/kurlar/today.xml


In [3]:
html = request.urlopen(url)
data = html.read()
import_time = datetime.now()

# READ DATA AND WRITE TO DATAFRAME

In [4]:
df = pd.read_xml(data)
df

,CrossOrder,Kod,CurrencyCode,Unit,Isim,CurrencyName,ForexBuying,ForexSelling,BanknoteBuying,BanknoteSelling,CrossRateUSD,CrossRateOther
0,0,USD,USD,1,ABD DOLARI,US DOLLAR,18.24740,18.28020,18.2346,18.3077,NaN,NaN
1,1,AUD,AUD,1,AVUSTRALYA DOLARI,AUSTRALIAN DOLLAR,12.17070,12.25010,12.1147,12.3236,1.4958,NaN
2,2,DKK,DKK,1,DANİMARKA KRONU,DANISH KRONE,2.44290,2.45490,2.4412,2.4606,7.4579,NaN
3,9,EUR,EUR,1,EURO,EURO,18.19540,18.22810,18.1826,18.2555,NaN,0.99720
4,10,GBP,GBP,1,İNGİLİZ STERLİNİ,POUND STERLING,20.74970,20.85790,20.7352,20.8892,NaN,1.13910
5,3,CHF,CHF,1,İSVİÇRE FRANGI,SWISS FRANK,18.89910,19.02050,18.8708,19.0490,0.9633,NaN
6,4,SEK,SEK,1,İSVEÇ KRONU,SWEDISH KRONA,1.68550,1.70290,1.6843,1.7069,10.7800,NaN
7,6,CAD,CAD,1,KANADA DOLARI,CANADIAN DOLLAR,13.72420,13.78610,13.6734,13.8384,1.3278,NaN
8,11,KWD,KWD,1,KUVEYT DİNARI,KUWAITI DINAR,58.69690,59.46500,57.8165,60.3570,NaN,3.23490
9,7,NOK,NOK,1,NORVEÇ KRONU,NORWEGIAN KRONE,1.77660,1.78850,1.7753,1.7926,10.2500,NaN


# CREATE SQL TABLE

In [5]:
connection_obj = sqlite3.connect('test_database')
cursor_obj = connection_obj.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS DovizSQL")
table = """ CREATE TABLE DovizSQL (
            
            CrossOrder VARCHAR(255) NOT NULL,
            Kod VARCHAR(255) NOT NULL,
            CurrencyCode VARCHAR(255) NOT NULL,
            Unit VARCHAR(255) NOT NULL,
            Isim VARCHAR(255) NOT NULL,
            CurrencyName VARCHAR(255) NOT NULL,
            ForexBuying VARCHAR(255) NOT NULL,
            ForexSelling VARCHAR(255) NOT NULL,
            BanknoteBuying VARCHAR(255) NOT NULL,
            BanknoteSelling VARCHAR(255) NOT NULL,
            CrossRateUSD VARCHAR(255) NOT NULL,
            CrossRateOther VARCHAR(255) NOT NULL
        ); """
 
cursor_obj.execute(table)

In [6]:
#conn = sqlite3.connect('DovizSQL')
c = connection_obj.cursor()

c.execute('CREATE TABLE IF NOT EXISTS DovizSQL (text)')
connection_obj.commit()

df.to_sql('DovizSQL', connection_obj, if_exists='replace', index = False)

23

In [7]:
c.execute('''  
SELECT * FROM DovizSQL
          ''')

for x in c.fetchall():
    print(x)
    

(0, 'USD', 'USD', 1, 'ABD DOLARI', 'US DOLLAR', 18.2474, 18.2802, 18.2346, 18.3077, None, None)
(1, 'AUD', 'AUD', 1, 'AVUSTRALYA DOLARI', 'AUSTRALIAN DOLLAR', 12.1707, 12.2501, 12.1147, 12.3236, 1.4958, None)
(2, 'DKK', 'DKK', 1, 'DANİMARKA KRONU', 'DANISH KRONE', 2.4429, 2.4549, 2.4412, 2.4606, 7.4579, None)
(9, 'EUR', 'EUR', 1, 'EURO', 'EURO', 18.1954, 18.2281, 18.1826, 18.2555, None, 0.9972)
(10, 'GBP', 'GBP', 1, 'İNGİLİZ STERLİNİ', 'POUND STERLING', 20.7497, 20.8579, 20.7352, 20.8892, None, 1.1391)
(3, 'CHF', 'CHF', 1, 'İSVİÇRE FRANGI', 'SWISS FRANK', 18.8991, 19.0205, 18.8708, 19.049, 0.9633, None)
(4, 'SEK', 'SEK', 1, 'İSVEÇ KRONU', 'SWEDISH KRONA', 1.6855, 1.7029, 1.6843, 1.7069, 10.78, None)
(6, 'CAD', 'CAD', 1, 'KANADA DOLARI', 'CANADIAN DOLLAR', 13.7242, 13.7861, 13.6734, 13.8384, 1.3278, None)
(11, 'KWD', 'KWD', 1, 'KUVEYT DİNARI', 'KUWAITI DINAR', 58.6969, 59.465, 57.8165, 60.357, None, 3.2349)
(7, 'NOK', 'NOK', 1, 'NORVEÇ KRONU', 'NORWEGIAN KRONE', 1.7766, 1.7885, 1.7753, 

# SUMMARY

In [8]:
date_alert = "The date of importing the today.xml file is " + str(import_time)

In [9]:
currency_codes = []
c.execute('''  
SELECT CurrencyCode FROM DovizSQL
          ''')
for data in c.fetchall():
    currency_codes.append(data[0])
    
currency_names = []
c.execute('''  
SELECT CurrencyName FROM DovizSQL
          ''')
for data in c.fetchall():
    currency_names.append(data[0])
    
forex_buying = []
c.execute('''  
SELECT ForexBuying FROM DovizSQL
          ''')
for data in c.fetchall():
    forex_buying.append(data[0])
    
forex_selling = []
c.execute('''  
SELECT ForexSelling FROM DovizSQL
          ''')
for data in c.fetchall():
    forex_selling.append(data[0])

In [10]:
print("\n" + date_alert)
print("\nCurrencyCode|CurrencyName|ForexBuying|ForexSelling\n")
for i in range(len(currency_codes)):
    print(currency_codes[i] + " | ", end="")
    print(currency_names[i] + " | ", end="")
    print(forex_buying[i], end="")
    print(" | ", end="")
    print(forex_selling[i], end="")
    print(" ")


The date of importing the today.xml file is 2022-09-18 10:58:44.344952

CurrencyCode|CurrencyName|ForexBuying|ForexSelling

USD | US DOLLAR | 18.2474 | 18.2802 
AUD | AUSTRALIAN DOLLAR | 12.1707 | 12.2501 
DKK | DANISH KRONE | 2.4429 | 2.4549 
EUR | EURO | 18.1954 | 18.2281 
GBP | POUND STERLING | 20.7497 | 20.8579 
CHF | SWISS FRANK | 18.8991 | 19.0205 
SEK | SWEDISH KRONA | 1.6855 | 1.7029 
CAD | CANADIAN DOLLAR | 13.7242 | 13.7861 
KWD | KUWAITI DINAR | 58.6969 | 59.465 
NOK | NORWEGIAN KRONE | 1.7766 | 1.7885 
SAR | SAUDI RIYAL | 4.8561 | 4.8648 
JPY | JAPENESE YEN | 12.7007 | 12.7848 
BGN | BULGARIAN LEV | 9.2664 | 9.3877 
RON | NEW LEU | 3.674 | 3.7221 
RUB | RUSSIAN ROUBLE | 0.30207 | 0.30602 
IRR | IRANIAN RIAL | 0.0432 | 0.04377 
CNY | CHINESE RENMINBI | 2.5865 | 2.6204 
PKR | PAKISTANI RUPEE | 0.0766 | 0.0776 
QAR | QATARI RIAL | 4.9628 | 5.0277 
KRW | SOUTH KOREAN WON | 0.01304 | 0.01321 
AZN | AZERBAIJANI NEW MANAT | 10.6799 | 10.8196 
AED | UNITED ARAB EMIRATES DIRHAM | 4